In [ ]:
"""
This script generates the subplots for figure 8
These are generated using the timeseries outputs from the HGTM
"""

In [9]:
import netCDF4 as nc
import numpy as np
from matplotlib import pyplot as plt

In [10]:
diameter_nc_ffn = '/home/meso/Insync/onedrive/sync/papers/hail-xsec_paper/hgtm-output/hgtm_growth_timeseries.nc'
regime_nc_ffn = '/home/meso/Insync/onedrive/sync/papers/hail-xsec_paper/hgtm-output/hgtm_growth_regime.nc'
output_file_folder = '/home/meso/Insync/onedrive/sync/papers/hail-xsec_paper/hgtm-output/xsecs_images'

In [15]:
#read required datasets
with nc.Dataset(diameter_nc_ffn) as ncid:
    diameters = ncid['GROWTH_TIMESERIES_ALL'][:] #diameters (in mm)
with nc.Dataset(regime_nc_ffn) as ncid:
    regimes = ncid['GROWTH_REGIME_ALL'][:] #regime, where 1 is wet growth and 0 is dry growth

In [16]:
n_hail = np.shape(diameters)[1]

In [25]:
#loop for each hailstone
for hail_idx in range(n_hail):
    print('plotting hailstone:', hail_idx+1)
    #extract dataset
    hail_diameter_ts = diameters[:,hail_idx]
    hail_regime_ts = regimes[:,hail_idx]
    nan_mask = np.isnan(hail_diameter_ts)
    hail_radius_ts = hail_diameter_ts[~nan_mask]/2
    hail_regime_ts = hail_regime_ts[~nan_mask]
    
    
    # set up figure
    fig = plt.figure(figsize=[10, 10])
    fig.set_facecolor("k")
    ax = plt.subplot(111)
    ax.set_aspect("equal")
    ax.set_facecolor("k")
    
    # draw entire hail
    max_radius = np.max(hail_radius_ts)
    background = plt.Circle((0, 0), max_radius, color=[0.5, 0.5, 0.5])
    ax.add_artist(background)
    
    #for each timestep
    for i in range(len(hail_radius_ts)):
        #add layers in reverse
        if hail_regime_ts[-i] == +1:
            #wet growth
            layer = plt.Circle((0, 0), hail_radius_ts[-i], color=[0.5, 0.5, 0.5])
        else:
            #dry growth
            layer = plt.Circle((0, 0), hail_radius_ts[-i], color=[1.0, 1.0, 1.0])
        ax.add_artist(layer)
    
    # draw embryo
    print(hail_radius_ts[0:2], hail_regime_ts[0:2])
    embryo = plt.Circle((0, 0), hail_radius_ts[0], color=[1.0, 1.0, 1.0])
    ax.add_artist(embryo)
    
    # draw grids on plot
    grid_coord = [-35, -25, -15, -5, 5, 15, 25, 35]
    for i in grid_coord:
        plt.plot([i, i], [-40, 40], "y:", lw=2)
        plt.plot([-40, 40], [i, i], "y:", lw=2)
    ax.set_xlim((-40, 40))
    ax.set_ylim((-40, 40))
    
    # export plot
    ax.set_axis_off()
    plt.savefig(
        f"{output_file_folder}/hgtm_eq_{hail_idx+1:02}.png",
        bbox_inches="tight",
        pad_inches=0,
    )
    plt.clf()
    plt.close("all")

plotting hailstone: 1
[2.5 2.499963783346112] [-1. -1.]
plotting hailstone: 2
[2.5 2.49996902785651] [-1. -1.]
plotting hailstone: 3
[2.5 2.5038157505853817] [-1. -1.]
plotting hailstone: 4
[2.5 2.499940858906215] [-1. -1.]
plotting hailstone: 5
[2.5 2.5023667310414686] [1. 1.]
plotting hailstone: 6
[2.5 2.5] [1. 1.]
plotting hailstone: 7
[2.5 2.5000891395166924] [-1. -1.]
plotting hailstone: 8
[2.5 2.4999638329192235] [-1. -1.]
plotting hailstone: 9
[2.5 2.4999800940283072] [-1. -1.]
plotting hailstone: 10
[2.5 2.502569965417518] [1. 1.]
plotting hailstone: 11
[2.5 2.4999408681067674] [-1. -1.]
plotting hailstone: 12
[2.5 2.499952677717418] [-1. -1.]
plotting hailstone: 13
[2.5 2.5] [1. 1.]
plotting hailstone: 14
[2.5 2.501784428448765] [-1. -1.]
plotting hailstone: 15
[2.5 2.500002493567485] [-1. -1.]
plotting hailstone: 16
[2.5 2.499973147940263] [-1. -1.]
plotting hailstone: 17
[2.5 2.49995262916667] [-1. -1.]
plotting hailstone: 18
[2.5 2.499935903299109] [-1. -1.]
plotting hailst